<a href="https://colab.research.google.com/github/Aydin-ab/CV_DMTet/blob/main/DMTet_Tut_(David).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies: Kaolin and PyTorch3D

In [1]:
from google.colab import drive, auth
drive.mount('/content/drive')
from pathlib import Path
INSTALL_PATH = Path("/content/drive/MyDrive/CV_DMTet/")
%cd  $INSTALL_PATH
%env XDG_CACHE_HOME=/content/drive/MyDrive/CV_DMTet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CV_DMTet
env: XDG_CACHE_HOME=/content/drive/MyDrive/CV_DMTet


In [2]:
# reinstall cython, install usd-core (for 3D rendering), and clone into kaolin repo
!pip uninstall Cython --yes
import torch
# if not torch.__version__.startswith("1.12"):
#   !pip uninstall torch --yes
# !pip install torch==1.12 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install  Cython==0.29.20  --quiet
!pip install  usd-core --quiet


Found existing installation: Cython 0.29.20
Uninstalling Cython-0.29.20:
  Successfully uninstalled Cython-0.29.20


In [3]:
# installing kaolin and check version
%env IGNORE_TORCH_VER=1
%env KAOLIN_INSTALL_EXPERIMENTAL=1
KAOLIN_PATH = INSTALL_PATH / "kaolin"
%cd $INSTALL_PATH
!if [ ! -d $KAOLIN_PATH ]; then git clone --recursive https://github.com/NVIDIAGameWorks/kaolin; fi;
%cd $KAOLIN_PATH
SETUP_CHECK = KAOLIN_PATH / "kaolin" / "version.py"
!echo Checking if $SETUP_CHECK exists
!if [ ! -f $SETUP_CHECK ]; then python setup.py develop; fi;
# !python -c "import kaolin; print(kaolin.__version__)"

env: IGNORE_TORCH_VER=1
env: KAOLIN_INSTALL_EXPERIMENTAL=1
/content/drive/MyDrive/CV_DMTet
/content/drive/MyDrive/CV_DMTet/kaolin
Checking if /content/drive/MyDrive/CV_DMTet/kaolin/kaolin/version.py exists


In [4]:
# !python setup.py install_lib install_scripts build
!python setup.py develop

setup.py:36: UserWarning: Kaolin is compatible with PyTorch >=1.6.0, <=1.13.0, but found version 1.13.0+cu116. Continuing with the installed version as IGNORE_TORCH_VER is set.
  warnings.warn(
Compiling kaolin/cython/ops/mesh/triangle_hash.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
Compiling kaolin/cython/ops/conversions/mise.pyx because it depends on /usr/local/lib/python3.8/dist-packages/Cython/Includes/libcpp/vector.pxd.
[1/2] Cythonizing kaolin/cython/ops/conversions/mise.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/MyDrive/CV_DMTet/kaolin/kaolin/cython/ops/conversions/mise.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing kaolin/cython/ops/mesh/triangle_hash.pyx
/usr/local/lib/python3.8/dist-packages/Cython/Compiler/Main.py:369: FutureWa

# Import packages

In [5]:
import numpy as np
import torch
import kaolin
import sys
import os

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        print(f"version_str : {version_str}")
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

from kaolin.ops.conversions import (
    trianglemeshes_to_voxelgrids,
    marching_tetrahedra,
    voxelgrids_to_cubic_meshes,
    voxelgrids_to_trianglemeshes,
)

from kaolin.ops.mesh import (
    index_vertices_by_faces
)

from kaolin.io.shapenet import (
    ShapeNetV2
)

from kaolin.metrics.trianglemesh import (
    point_to_mesh_distance,

)

from torch.utils.data import DataLoader

# add path for demo utils functions 
sys.path.append(os.path.abspath(''))
sys.path.append('/content/drive/MyDrive/CV_DMTet/')

# Import Dataset: Subset of ShapeNetV2

In [6]:
import pytorch3d

In [7]:
# state_dict = torch.load('/content/drive/MyDrive/CV_DMTet/shapenet.pvcnn.c1.pth.tar')
# state_dict.keys()

In [8]:
sys.path.append(INSTALL_PATH / "examples")
sys.path.append(INSTALL_PATH / "examples" / "tutorial")

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [10]:
# # path to the point cloud to be reconstructed
# pcd_path = KAOLIN_PATH / "examples/samples/bear_pointcloud.usd"
# # path to the output logs (readable with the training visualizer in the omniverse app)
logs_path = '/content/drive/MyDrive/CV_DMTet/Logs'

# # We initialize the timelapse that will store USD for the visualization apps
timelapse = kaolin.visualize.Timelapse(logs_path)

#Load Tetrahedral grid

DMTet starts from a uniform tetrahedral grid of predefined resolution, and uses a network to predict the SDF value as well as deviation vector at each grid vertex.

Here we load the pre-generated tetrahedral grid using Quartet at resolution 128, which has roughly the same number of vertices as a voxel grid of resolution 65. We use a simple MLP + positional encoding to predict the SDF and deviation vectors in DMTet, and initialize the encoded SDF to represent a sphere.

In [11]:
# Uniform Tetrahedral Grid
tets_verts = torch.tensor(np.load(KAOLIN_PATH / "examples/samples/128_verts.npz")['data'], dtype=torch.float, device=device)
tets = torch.tensor(([np.load(KAOLIN_PATH / 'examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)
print(tets_verts, tets)


tensor([[ 0.5000,  0.5000,  0.4844],
        [ 0.4844,  0.5000,  0.4922],
        [ 0.4922,  0.4844,  0.4844],
        ...,
        [-0.1719, -0.5000,  0.4766],
        [-0.1562, -0.5000,  0.4688],
        [-0.1562, -0.4922,  0.4688]], device='cuda:0') tensor([[     0,      1,      2,      3],
        [     2,      3,      1,      4],
        [     5,      3,      0,      2],
        ...,
        [277409, 272920, 272914, 272919],
        [272919, 277409, 272920, 274866],
        [277409, 277400, 272920, 274866]], device='cuda:0')


<ipython-input-11-e89bff709631>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tets = torch.tensor(([np.load(KAOLIN_PATH / 'examples/samples/128_tets_{}.npz'.format(i))['data'] for i in range(4)]), dtype=torch.long, device=device).permute(1,0)


# Loading from ShapeNet

In [12]:
# Not using for now, using libigl tutorial x-cylinder below
SHAPENET_PATH = "/content/drive/MyDrive/CV_DMTet/Core"
#SHAPENET_PATH = "/content/drive/MyDrive/FALL 2022/Computer Vision/Project/Core"
SYNSETS_IDS = ['02747177', '02801938', '02808440', '02818832', '02828884', '02871439', '02876657', '02880940', '02924116', '02933112']
shapenet_train = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True)
shapenet_test = ShapeNetV2(SHAPENET_PATH, categories=SYNSETS_IDS, output_dict=True, train=False)

In [13]:
mesh = shapenet_train[10]['mesh'] # change the index here for different models
gt_verts = mesh[0].to(device)
gt_faces = mesh[1].to(device)
print(gt_verts.shape)
timelapse.add_mesh_batch(
    category='gt',
    vertices_list=[gt_verts.cpu()],
    faces_list=[gt_faces.cpu()]
)
gt_sample = int(0.99*gt_verts.shape[0])

torch.Size([6038, 3])


In [14]:
# Clone sample meshes from libigl tutorial 1x
LIBIGL_TUTORIAL_DATA = INSTALL_PATH / "libigl-tutorial-data"
print(f"LIBIGL_TUTORIAL_DATA: {LIBIGL_TUTORIAL_DATA}")
%cd $INSTALL_PATH
!if [ ! -d $LIBIGL_TUTORIAL_DATA ]; then git clone --recursive https://github.com/libigl/libigl-tutorial-data.git; fi;
%cd $KAOLIN_PATH

LIBIGL_TUTORIAL_DATA: /content/drive/MyDrive/CV_DMTet/libigl-tutorial-data
/content/drive/MyDrive/CV_DMTet
/content/drive/MyDrive/CV_DMTet/kaolin


In [15]:
# Load cylinder (along x-axis) mesh (V,F) = (42,80)
# mesh = kaolin.io.obj.import_mesh(LIBIGL_TUTORIAL_DATA / "arm.obj") #xcylinder.obj
# gt_verts = mesh[0].to(device)
# gt_faces = mesh[1].to(device)
# timelapse.add_mesh_batch(
#     category='gt',
#     vertices_list=[gt_verts.cpu()],
#     faces_list=[gt_faces.cpu()]
# )
# gt_sample = int(0.99*gt_verts.shape[0])

In [16]:
points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, gt_sample)[0][0]
center = (points.max(0)[0] + points.min(0)[0]) / 2
max_l = (points.max(0)[0] - points.min(0)[0]).max()
points = ((points - center) / max_l)* 0.9
points.to(device)
timelapse.add_pointcloud_batch(category='input',
                               pointcloud_list=[points.cpu()], points_type = "usd_geom_points")

# Convert model to watertight meshes

We used a voxelization with resolution of 64 to predict the sdf and extract surface. 

In [17]:
# wt_grid
# Convert mesh (V,F) to a voxel grid 
# voxels = kaolin.ops.conversions.trianglemeshes_to_voxelgrids(
#     vertices=mesh_verts.unsqueeze(0).to(device),
#     faces=mesh_faces.to(device),
#     resolution=16
# )

# # Convert the voxel grid back into a mesh as GroundTruth
# # I am not sure this step is necessary
# # The intention is to have mesh_vertices, mesh_faces for the voxel grid for loss function
# wt_verts, wt_faces = kaolin.ops.conversions.voxelgrids_to_cubic_meshes(voxels)
# wt_verts, wt_faces = wt_verts[0], wt_faces[0]

# max_gt_len = (mesh_verts.max(0)[0] - mesh_faces.min(0)[0]).max()
# max_vox_len = (wt_verts.max(0)[0] - wt_verts.min(0)[0]).max()
# scale = max_gt_len / max_vox_len
# center = (wt_verts.max(0)[0] + wt_verts.min(0)[0]) / 2
# wt_verts = ((wt_verts - center) * scale)

# timelapse.add_mesh_batch(
#     category='watertight_test',
#     vertices_list=[wt_verts.cpu()],
#     faces_list=[wt_faces.cpu()]
# )

In [18]:
# Using Kaolin bear point cloud data (PCD) right now since tutorial uses PCD
# pcd_path = KAOLIN_PATH / "examples/samples/bear_pointcloud.usd"
# points = kaolin.io.usd.import_pointclouds(str(pcd_path))[0].points.to(device)
# if points.shape[0] > 100000:
#     idx = list(range(points.shape[0]))
#     np.random.shuffle(idx)
#     idx = torch.tensor(idx[:100000], device=points.device, dtype=torch.long)    
#     points = points[idx]

# # The reconstructed object needs to be slightly smaller than the grid to get watertight surface after MT.
# # The idea is that we want our point cloud to expand since it will converge faster if all face expansions are outward.
# center = (points.max(0)[0] + points.min(0)[0]) / 2
# max_l = (points.max(0)[0] - points.min(0)[0]).max()
# points = ((points - center) / max_l)* 0.9
# timelapse.add_pointcloud_batch(category='input',
#                                pointcloud_list=[points.cpu()], points_type = "usd_geom_points")

kaolin.ops.conversions.voxelgrids_to_cubic_meshes(

Convert voxelgrids to meshes by replacing each occupied voxel with a cuboid mesh (unit cube). Each cube has 8 vertices and 6 (for quadmesh) or 12 faces (for triangular mesh). Internal faces are ignored. If is_trimesh==True, this function performs the same operation as “Cubify” defined in the ICCV 2019 paper “Mesh R-CNN”: https://arxiv.org/abs/1906.02739.

Parameters
voxelgrids (torch.Tensor) – binary voxel array, of shape .

is_trimesh (optional, bool) – if True, the outputs are triangular meshes. Otherwise quadmeshes are returned. Default: True.

Returns
The list of vertices for each mesh.

The list of faces for each mesh.

Return type
(list[torch.Tensor], list[torch.LongTensor])

kaolin.ops.conversions.marching_tetrahedra(vertices, tets, sdf, return_tet_idx=False)¶
Convert discrete signed distance fields encoded on tetrahedral grids to triangle meshes using marching tetrahedra algorithm as described in An efficient method of triangulating equi-valued surfaces by using tetrahedral cells. The output surface is differentiable with respect to input vertex positions and the SDF values. For more details and example usage in learning, see Deep Marching Tetrahedra: a Hybrid Representation for High-Resolution 3D Shape Synthesis NeurIPS 2021.


# SDF model

We follow the paper recommandation and use a four-layer
MLPs with hidden dimensions 256, 256, 128 and 64, respectively

In [19]:
# Since we skip PVCNN, input dimension is just the coordinates of each grid

SDF_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 256, 128, 64],
    'output_dim' : 1, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
    'multires': 2
}

ENCODER_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 800, 1600, 1600],
    'output_dim' : 832, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
}

lr = 0.001
laplacian_weight = 0.1
iterations = 2000
save_every = 100
multires = 2
grid_res = 128

In [20]:
def get_pred_sdfs(model, tets_verts):
  model.eval()

  with torch.no_grad():
    pred, f_vs = model(tets_verts)
    pred_sdfs = pred[0]

  return pred_sdfs, f_vs

# Get ground truth sdf from input verts and faces
# input shape: [batch_size, num_vertices, 3], [num_faces, 4], [batch_size, num_points, 3]
# output shape: [num_points, 1]
def get_gt_sdfs(gt_verts, gt_faces, points, f=None):

  if f == None:
    f = index_vertices_by_faces(gt_verts, gt_faces)
  d,_,_ = point_to_mesh_distance(points, f)

  s = kaolin.ops.mesh.check_sign(gt_verts, gt_faces, points)
  d[s==True] *= -1

  return d.squeeze(0).unsqueeze(1)

In [42]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from pytorch3d.utils import ico_sphere

# Comes from the DMTet tutorial 
# MLP + Positional Encoding
class Decoder(torch.nn.Module):
    def __init__(self, input_dims = 3, internal_dims = 64, output_dims = 4, hidden = 5, multires = 2):
        super().__init__()
        self.embed_fn = None
        if multires > 0:
            embed_fn, input_ch = get_embedder(multires)
            self.embed_fn = embed_fn
            input_dims = input_ch

        # Hidden layers
        self.hiddens = nn.ModuleList()
        net = (torch.nn.Linear(input_dims, internal_dims, bias=False), torch.nn.ReLU())
        for i in range(hidden-1):
            net = net + (torch.nn.Linear(internal_dims, internal_dims, bias=False), torch.nn.ReLU())
        self.net = torch.nn.Sequential(*net)
        self.output = torch.nn.Linear(internal_dims, output_dims, bias=False)

    def forward(self, p):
        if self.embed_fn is not None:
            p = self.embed_fn(p)
        p = self.net(p)
        out = self.output(p)
        return out, p
    
    def pre_train_sphere(self, iter):    
        print ("Initialize SDF to sphere")
        loss_fn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)
        

        for i in tqdm(range(iter)):
            p = torch.rand((1024, 3), device='cuda') - 0.5
            ref_value  = torch.sqrt(((p+.5)**2).sum(-1)) - 0.3
            output, _ = self(p)
            loss = loss_fn(output[...,0], ref_value)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Pre-trained MLP", loss.item())

# Positional Encoding from https://github.com/yenchenlin/nerf-pytorch/blob/1f064835d2cca26e4df2d7d130daa39a8cee1795/run_nerf_helpers.py
class Embedder:
    def __init__(self, **kwargs):
        self.kwargs = kwargs
        self.create_embedding_fn()
        
    def create_embedding_fn(self):
        embed_fns = []
        d = self.kwargs['input_dims']
        out_dim = 0
        if self.kwargs['include_input']:
            embed_fns.append(lambda x : x)
            out_dim += d
            
        max_freq = self.kwargs['max_freq_log2']
        N_freqs = self.kwargs['num_freqs']
        
        if self.kwargs['log_sampling']:
            freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)
        else:
            freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)
            
        for freq in freq_bands:
            for p_fn in self.kwargs['periodic_fns']:
                embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
                out_dim += d
                    
        self.embed_fns = embed_fns
        self.out_dim = out_dim
        
    def embed(self, inputs):
        return torch.cat([fn(inputs) for fn in self.embed_fns], -1)

def get_embedder(multires):
    embed_kwargs = {
                'include_input' : True,
                'input_dims' : 3,
                'max_freq_log2' : multires-1,
                'num_freqs' : multires,
                'log_sampling' : True,
                'periodic_fns' : [torch.sin, torch.cos],
    }
    
    embedder_obj = Embedder(**embed_kwargs)
    embed = lambda x, eo=embedder_obj : eo.embed(x)
    return embed, embedder_obj.out_dim

class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()

        self.input_dim = config['input_dim']
        self.hidden_dims  = config['hidden_dims']
        self.output_dim = config['output_dim']

        # Hidden layers
        self.hiddens = nn.ModuleList()
        in_dim = self.input_dim
        for k in range(len(self.hidden_dims)):
            self.hiddens.append(nn.Linear(in_dim, self.hidden_dims[k]))
            in_dim = self.hidden_dims[k]

        # Output layer
        self.output_layer = torch.nn.Linear(self.hidden_dims[-1], self.output_dim)

    def forward(self, x):
        for hidden in self.hiddens :
            x = F.relu(hidden(x))
        output = self.output_layer(x) # No activation (linear) cuz we do regression

        return output, x # Return output + last feature layer vector

    def pre_train_sphere(self, iter):    
        print ("Initialize SDF to sphere")
        loss_fn = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(list(self.parameters()), lr=1e-4)
        

        for i in tqdm(range(iter)):
            p = torch.rand((1024,self.input_dim), device=device) - 0.5
            ref_value  = torch.sqrt(((p+.5)**2).sum(-1)) - 0.3
            output, _ = self(p)
            loss = loss_fn(output[...,0], ref_value)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Pre-trained MLP", loss.item())

In [22]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [23]:
from tqdm import tqdm
# # Initialize model and create optimizer
sdf_model = Decoder(multires=2).to(device)
sdf_model.pre_train_sphere(1000)
# print(sdf_model)
# print('\n\n')
# summary(sdf_model, input_size=tets_verts.shape)

Initialize SDF to sphere


100%|██████████| 1000/1000 [00:03<00:00, 258.54it/s]

Pre-trained MLP 7.655046647414565e-05


In [24]:
encoder_model = MLP(SDF_MLP_CONFIG).to(device)
print(encoder_model)
# print('\n\n')
summary(encoder_model, input_size=tets_verts.shape)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
  )
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1          [-1, 277410, 256]           1,024
            Linear-2          [-1, 277410, 256]          65,792
            Linear-3          [-1, 277410, 128]          32,896
            Linear-4           [-1, 277410, 64]           8,256
            Linear-5            [-1, 277410, 1]              65
Total params: 108,033
Trainable params: 108,033
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 3.17
Forward/backward pass size (MB): 1492.11
Pa

Select SDF Model
* sdf_model is standard MLP (all hidden dims are same) + f_v + comes with encoder deformer
* encoder_model is from DMTet and encodes layers + f_v + no deformer

In [25]:
MODEL = encoder_model

Little Test:

In [26]:
# signed distance fields encoded on tetrahedral grid

# pred_sdfs dim = (tets_vertices.shape[0])
# f_vs ie f_v feature vector

# sdf, delta x_i, delta y_i, delta z_i, 
# sdf = binary occupancy of tetrahedron 
pred, _ = MODEL(tets_verts)
print(f'pred shape is : {pred.shape}')

# pred_sdfs, f_vs = sdf_model(tets_verts)

print(f'Input grid shape is : {tets_verts.shape}')
# print(f'Output shape of the predicted SDFs should be {tets_verts.shape[0], 1} and it actually is {tuple(pred_sdfs.shape)}')
# print(f'Output shape of the feature vectors f_vs should be {tets_verts.shape[0], 64} and it actually is {tuple(f_vs.shape)}')
# print(pred_sdfs)

pred shape is : torch.Size([277410, 1])
Input grid shape is : torch.Size([277410, 3])


# Set up Optimizer

In [27]:
sdf_vars = [p for _, p in MODEL.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

# Training

In [28]:
# # # # takes in a module and applies the specified weight initialization
# def weights_init_normal(m):
#     '''Takes in a module and initializes all linear layers with weight
#         values taken from a normal distribution.'''
#     classname = m.__class__.__name__
#     # for every Linear layer in a model
#     if classname.find('Linear') != -1:
#         y = m.in_features
#     # m.weight.data shoud be taken from a normal distribution
#         m.weight.data.normal_(0.0,1/np.sqrt(y))
#         m.weight.data = torch.randint(-1,2,m.weight.shape)
#     # m.bias.data should be 0
#         if m.bias is not None:
#           m.bias.data.fill_(0)


In [29]:
# sdf_model.apply(weights_init_normal)

In [30]:
# sdf_model.net[10].weight.data

In [53]:
# Laplacian regularization using umbrella operator (Fujiwara / Desbrun).
# https://mgarland.org/class/geom04/material/smoothing.pdf
from pytorch3d import loss
def laplace_regularizer_const(pred_mesh_verts, pred_mesh_faces):
    term = torch.zeros_like(pred_mesh_verts, device=device)
    norm = torch.zeros_like(pred_mesh_verts[..., 0:1], device=device)

    v0 = pred_mesh_verts[pred_mesh_faces[:, 0], :]
    v1 = pred_mesh_verts[pred_mesh_faces[:, 1], :]
    v2 = pred_mesh_verts[pred_mesh_faces[:, 2], :]

    term.scatter_add_(0, pred_mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, pred_mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, pred_mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, pred_mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, pred_mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, pred_mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def loss_f(pred_mesh_verts, pred_mesh_faces, pts, it):
    num_samples = int(0.3 * pred_mesh_verts.shape[0])
    pred_points = kaolin.ops.mesh.sample_points(pred_mesh_verts.unsqueeze(0), 
                                                pred_mesh_faces, num_samples)[0][0]
    
    chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), pts.unsqueeze(0)).mean()
    # chamfer.clone().detach().requires_grad_(True)
    if it > iterations//2:
        lap = laplace_regularizer_const(pred_mesh_verts, pred_mesh_faces)
        return chamfer + lap * laplacian_weight
    return chamfer 


def sdf_train(iterations, model, optimizer, scheduler):
  # Set to training mode
  model.train()

  for it in range(iterations):
      pred, _ = model(tets_verts)
      
      # Replace pred_sdfs with the new SDF
      # Replace deform with surface refinement 

      pred_sdfs, deform = pred[:,0], pred[:,1:]
      verts_deformed = tets_verts + torch.tanh(deform) / grid_res # constraint deformation to avoid flipping tets
      pred_mesh_verts, pred_mesh_faces = marching_tetrahedra(verts_deformed.unsqueeze(0), tets, pred_sdfs.unsqueeze(0))
      pred_mesh_verts, pred_mesh_faces = pred_mesh_verts[0], pred_mesh_faces[0]

      loss = loss_f(pred_mesh_verts, pred_mesh_faces, points, it)
      
      optimizer.zero_grad()
      torch.autograd.set_detect_anomaly(True)
      loss.backward(retain_graph=True)
      optimizer.step()
      scheduler.step()

      if (it) % save_every == 0 or it == (iterations - 1): 
          print ('Iteration {} - loss: {}'.format(it, loss))
          # save reconstructed mesh
          timelapse.add_mesh_batch(
              iteration=it+1,
              category='extracted_mesh',
              vertices_list=[pred_mesh_verts.cpu()],
              faces_list=[pred_mesh_faces.cpu()]
          )

def encoder_train(iterations, model, optimizer, scheduler, tv = None):
  # Set to training mode
  model.train()

  if tv is None:
    tv = tets_verts

  for it in range(iterations):
      pred_sdfs, _ = model(tv)
      
      gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0))

      loss = F.mse_loss(pred_sdfs, gt_sdfs) 
      
      optimizer.zero_grad()
      torch.autograd.set_detect_anomaly(True)
      loss.backward(retain_graph=True)
      optimizer.step()
      scheduler.step()

      if (it) % save_every == 0 or it == (iterations - 1): 
          print ('Iteration {} - loss: {}'.format(it, loss))
          # save reconstructed mesh
          # timelapse.add_mesh_batch(
          #     iteration=it+1,
          #     category='extracted_mesh',
          #     vertices_list=[pred_mesh_verts.cpu()],
          #     faces_list=[pred_mesh_faces.cpu()]
          # )


In [32]:
MODEL.pre_train_sphere(1000)
sdf_vars = [p for _, p in MODEL.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time
# ~12 min. Speed up?
encoder_train(iterations, MODEL, sdf_optimizer, sdf_scheduler)
# sdf_train(iterations, MODEL, sdf_optimizer, sdf_scheduler)

Initialize SDF to sphere


100%|██████████| 1000/1000 [00:03<00:00, 319.68it/s]


Pre-trained MLP 9.1721612989204e-06
Iteration 0 - loss: 0.4624950587749481
Iteration 100 - loss: 0.00012578710447996855
Iteration 200 - loss: 2.572278208390344e-05
Iteration 300 - loss: 1.6134466932271607e-05
Iteration 400 - loss: 1.3485100680554751e-05
Iteration 500 - loss: 1.1769434422603808e-05
Iteration 600 - loss: 1.0726682376116514e-05
Iteration 700 - loss: 9.74570957623655e-06
Iteration 800 - loss: 9.058786417881493e-06
Iteration 900 - loss: 8.547104698664043e-06


KeyboardInterrupt: ignored

 # Surface refinement utils

In [ ]:
# Get edges lists of shape [E, 2] from face list of shape [V, 4]

def get_edges(input):
  c = torch.combinations(torch.arange(input.size(1)), r=2)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

# Extract tets under certain sdf restrictions:
# if thresh = 0, return all surface tetrahedrons
# if thresh > 0, return all tetrahedrons whose vertices' sdfs are all in the range [-thresh, thresh]

def extract_tet(tets, sdf, thresh, non_surf=False):

  assert thresh >= 0

  if thresh == 0:
    mask = sdf[tets] > 0
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[(t > 0) & (t < 4)]
  else:
    mask = (sdf[tets] >= -thresh) & (sdf[tets] <= thresh)
    mask_int = mask.squeeze(2).long()
    t = mask_int.sum(1)
    surf_tets = tets[t == 4]

  surf_tets_tuple = surf_tets.unique(return_inverse=True)
  surf_tets_idx, surf_tets = surf_tets_tuple[0], surf_tets_tuple[1]

  if non_surf:
    if thresh == 0:
      non_surf_tets = tets[~((t > 0) & (t < 4))]
    else:
      non_surf_tets = tets[t < 4]
    non_surf_tets_tuple = non_surf_tets.unique(return_inverse=True)
    non_surf_tets_idx, non_surf_tets = non_surf_tets_tuple[0], non_surf_tets_tuple[1]
    return surf_tets_idx, surf_tets, non_surf_tets_idx, non_surf_tets

  return surf_tets_idx, surf_tets

#Get output from initial MLP



In [ ]:
pred, f_vs = sdf_model(tets_verts)
pred_sdfs = pred[:,0]
d = get_gt_sdfs(gt_verts.unsqueeze(0).to(device), gt_faces.to(device), tets_verts.unsqueeze(0))
pred_sdfs, _ = get_pred_sdfs(sdf_model, tets_verts)

# Surface refinement model

In [ ]:
"""
Graph-res net:
Identify surface tetrahedral, build adj matrix, 
"""

from tqdm import tqdm

from torch import nn
from torch.nn import functional as F
from pytorch3d.ops import GraphConv

# a single res block layer with dimension 256 & 128
class GResBlock(nn.Module): 
    def __init__(self, in_dim, hidden_dim, activation=None):
        super(GResBlock, self).__init__()

        self.conv1 = GraphConv(in_dim, hidden_dim)
        self.conv2 = GraphConv(hidden_dim, in_dim)
        self.activation = F.relu if activation else None
    
    def forward(self, inputs):
        input, adj = inputs[0], inputs[1]
        x = self.conv1(input, adj)
        if self.activation:
          x = self.activation(x)
        x = self.conv2(x, adj)
        if self.activation:
          x = self.activation(input + x)
        
        return [x, adj]

class GBottleneck(nn.Module):
    def __init__(self, block_num, in_dim, hidden_dim, out_dim, activation=None):
        super(GBottleneck, self).__init__()

        resblock_layers = [GResBlock(in_dim=hidden_dim[0], hidden_dim=hidden_dim[1], activation=activation)
                          for _ in range(block_num)]
        self.blocks = nn.Sequential(*resblock_layers)
        self.conv1 = GraphConv(in_dim, hidden_dim[0])

        self.activation = F.relu if activation else None
    
    def forward(self, inputs, adj):
        x = self.conv1(inputs, adj)
        if self.activation:
          x = self.activation(x)
        x = self.blocks([x, adj])[0]
        if self.activation:
          x = self.activation(x)

        return x

class GCN_Res(nn.Module):
    def __init__(self, config):
        super(GCN_Res, self).__init__()

        self.in_dim = config['in_dim']
        self.hidden_dim = config['hidden_dim']
        self.out_dim = config['out_dim']
        self.activation = config['activation']
        self.mlp_hdim = config['mlp_hdim']
        self.mlp_odim = config['mlp_odim']

        self.gcn_res = nn.ModuleList([GBottleneck(2, self.in_dim, self.hidden_dim, self.out_dim, self.activation)])

        self.sdf_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 1, bias=False),
        )

        self.deform_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], 3, bias=False),
        )

        self.feature_mlp = nn.Sequential(
            nn.Linear(self.out_dim, self.mlp_hdim[0], bias=False),
            nn.Linear(self.mlp_hdim[0], self.mlp_hdim[1], bias=False),
            nn.Linear(self.mlp_hdim[1], self.mlp_hdim[1], bias=False),
        )
        


    def forward(self, inputs, adj):

        x = self.gcn_res[0](inputs, adj)

        sdf = self.sdf_mlp(x)
        deform = self.deform_mlp(x)
        deform = torch.tanh(deform)
        feature = self.feature_mlp(x)
        
        return sdf, deform, feature


In [ ]:
def laplace_regularizer_const(mesh_verts, mesh_faces):
    term = torch.zeros_like(mesh_verts)
    norm = torch.zeros_like(mesh_verts[..., 0:1])

    v0 = mesh_verts[mesh_faces[:, 0], :]
    v1 = mesh_verts[mesh_faces[:, 1], :]
    v2 = mesh_verts[mesh_faces[:, 2], :]

    term.scatter_add_(0, mesh_faces[:, 0:1].repeat(1,3), (v1 - v0) + (v2 - v0))
    term.scatter_add_(0, mesh_faces[:, 1:2].repeat(1,3), (v0 - v1) + (v2 - v1))
    term.scatter_add_(0, mesh_faces[:, 2:3].repeat(1,3), (v0 - v2) + (v1 - v2))

    two = torch.ones_like(v0) * 2.0
    norm.scatter_add_(0, mesh_faces[:, 0:1], two)
    norm.scatter_add_(0, mesh_faces[:, 1:2], two)
    norm.scatter_add_(0, mesh_faces[:, 2:3], two)

    term = term / torch.clamp(norm, min=1.0)

    return torch.mean(term**2)

def gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it):

    #surface alignment loss
    m = pytorch3d.structures.Meshes([mesh_verts, gt_verts], [mesh_faces, gt_faces])
  
    # pc = pytorch3d.ops.sample_points_from_meshes(m, num_samples=50000)
    if mesh_verts.shape[0] > 0:
      pred_points = kaolin.ops.mesh.sample_points(mesh_verts.unsqueeze(0), mesh_faces, gt_sample)[0][0]
      gt_points = kaolin.ops.mesh.sample_points(gt_verts.unsqueeze(0), gt_faces, gt_sample)[0][0]
      # pred_points,gt_points = pc[0], pc[1]
      chamfer = kaolin.metrics.pointcloud.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0), squared=False).mean()
    else:
      chamfer = 0


    #chamfer, p_norms = pytorch3d.loss.chamfer_distance(pred_points.unsqueeze(0), gt_points.unsqueeze(0), point_reduction='sum')
    # norm = torch.sum(1-torch.abs(p_norms))
    Lnorm = pytorch3d.loss.mesh_normal_consistency(m)
    if it > iterations//2:
      lap = laplace_regularizer_const(mesh_verts, mesh_faces)
      return 500*chamfer + lap * laplacian_weight + 1e-6*Lnorm
    return 500*chamfer + 1e-6*Lnorm 

# Train GCN Model for Surface Refinement

In [ ]:
CONFIG_GCNRES = {
    'in_dim': 68,
    'hidden_dim': [128, 256],
    'out_dim': 128,
    'activation': True,
    'mlp_hdim': [128,64],
    'mlp_odim': 68,
}

# Same set of Hyperparam is applied
lr = 1e-4
laplacian_weight = 0.1
gcn_iterations = 5000
save_every = 100
multires = 2
grid_res = 128

In [ ]:
from tqdm import tqdm

def gcn_pretrain(iterations, gcn_model, optimizer, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdf, thresh)
  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = pred_sdf[surf_tets_verts_idx]
  surf_tets_edges = torch.clone(get_edges(surf_tets_faces).to(device))
  
  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)

  timelapse.add_mesh_batch(
          category='tet_surface',
          vertices_list=[surf_tets_verts.cpu()],
          faces_list=[surf_tets_faces.cpu()]
      )

  for it in tqdm(range(iterations)):
    verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

    sdf, deform, fv = gcn_model(verts_f, surf_tets_edges)

    update_verts = surf_tets_verts + deform / grid_res
    update_sdfs = surf_sdfs + sdf

    gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_verts.unsqueeze(0), gt_f)

    sdf_loss = F.mse_loss(sdf, gt_sdfs - surf_sdfs, reduction='sum')

    optimizer.zero_grad()
    sdf_loss.backward(retain_graph=True)
    optimizer.step()

    if it == iterations-1:
      print('Iteration {} - loss: {}'.format(iterations, sdf_loss))

In [ ]:
"""
Identify surface Tetrahedral
input: f = [v, s, F_vol, f_64]
output: f = [delta v, delta s, f_64]

update v = v + delta v, s = s + delta s
generate new meshes by marching tet from v and s; calculate loss

loss = chamfer + sdf_loss + deform_loss
"""

def gcn_train(iterations, gcn_model, optimizer, scheduler, pred_sdf, tets_verts, tets, tets_verts_features, gt_verts, gt_faces, thresh):
  gcn_model.train()

  assert thresh >= 0

  """

  Firstly, based on predicted sdf value, extract out all surface tetrahedrons
  i.e. 1-3 vertices of tetrahedrons are with different sdf signs
  return value: 
  --> surf_tets_idx: origin index of vertices as in tet grid
  --> surf_tets_faces: reindexed faces list start from 0

  """

  surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, pred_sdf, thresh)
  surf_tets_edges = get_edges(surf_tets_faces).to(device)

  surf_tets_verts = torch.clone(tets_verts[surf_tets_verts_idx])

  # obtain per vertex features: 
  # sdf, position, feature vector

  surf_tets_verts_features = torch.clone(tets_verts_features[surf_tets_verts_idx])
  surf_sdfs = torch.clone(pred_sdf[surf_tets_verts_idx])
  surf_verts_f = torch.cat((surf_tets_verts, surf_sdfs, surf_tets_verts_features), dim=1)

  # calculate ground truth sdf value over all vertices in tet mesh

  gt_verts, gt_faces = gt_verts.to(device), gt_faces.to(device)

  gt_f = index_vertices_by_faces(gt_verts.unsqueeze(0), gt_faces)
  
  gt_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, tets_verts.unsqueeze(0), gt_f)

  for it in tqdm(range(iterations)):
    
    """
    
    Secondly, predict with a 2-layer GCN followed by 2-layer MLP
    output:
    --> [delta v, delta s, new f_s] \in R^68

    """

    sdf, deform, fv = gcn_model(surf_verts_f, surf_tets_edges)
    
    """

    Update surface position, sdf, and f_s

    """

    #updated sdf

    update_sdfs = torch.clone(pred_sdf)
    update_sdfs[surf_tets_verts_idx] += sdf
    

    #update vertices positions

    vd = deform
    update_tets_verts = torch.clone(tets_verts)
    update_tets_verts[surf_tets_verts_idx] += deform / grid_res

    #update vertices features

    vfvs = fv.detach()
    update_tets_f = torch.clone(tets_verts_features)
    update_tets_f[surf_tets_verts_idx] += vfvs

    #test
    MT_sdf = surf_sdfs + sdf
    MT_verts = surf_tets_verts + deform

    """

    Marching Tetrahedra based on new sdf value and deformed vertices in the tet grid

    """

    mesh_verts, mesh_faces = kaolin.ops.conversions.marching_tetrahedra(update_tets_verts.unsqueeze(0), tets, update_sdfs.squeeze(1).unsqueeze(0))
    mesh_verts, mesh_faces = mesh_verts[0], mesh_faces[0]

    """

    Compute Loss for First surface refinement: 
    Normal consistency + surface alignment + laplacian smooth + sdf L2-reg + deform L2-reg

    """

    # L2 sdf reg: 

    # s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_tets_verts[surf_tets_verts_idx].unsqueeze(0), gt_f)
    s_sdfs = get_gt_sdfs(gt_verts.unsqueeze(0), gt_faces, update_tets_verts.unsqueeze(0), gt_f)


    sdf_loss = F.mse_loss(update_sdfs, s_sdfs, reduction='mean')

    # #L2 deform reg

    deform_loss = F.mse_loss(update_tets_verts, tets_verts, reduction='mean')


    #surface alignment loss

    r_loss = gcn_loss(iterations, mesh_verts, mesh_faces, gt_verts, gt_faces, it)

    g_loss = r_loss + 0.4*sdf_loss + deform_loss 

    optimizer.zero_grad()
    torch.autograd.set_detect_anomaly(True)
    g_loss.backward(retain_graph=True)
    optimizer.step()
    scheduler.step()

    if (it) % save_every == 0 or it == (iterations - 1): 
      print ('Iteration {} - loss: {}, # of mesh vertices: {}, # of mesh faces: {}'.format(it, g_loss, mesh_verts.shape[0], mesh_faces.shape[0]))
      
      # save reconstructed mesh
      timelapse.add_mesh_batch(
          iteration=it+1,
          category='train_res',
          vertices_list=[mesh_verts.cpu()],
          faces_list=[mesh_faces.cpu()]
      )

In [ ]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)

print(refine_model)

pre_vars = [p for _, p in refine_model.named_parameters()]
pre_optimizer = torch.optim.Adam(pre_vars, lr=lr)


In [ ]:
gcn_pretrain(500, refine_model, pre_optimizer, d, tets_verts, tets, f_vs, gt_verts, gt_faces, 0.001)

In [ ]:
refine_vars = [p for _, p in refine_model.named_parameters()]
refine_optimizer = torch.optim.Adam(pre_vars, lr=lr)
refine_scheduler = torch.optim.lr_scheduler.LambdaLR(pre_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time
     

In [ ]:
pred_sdfs, _ = get_pred_sdfs(sdf_model, tets_verts)

In [ ]:
torch.cuda.empty_cache()
gcn_train(1000, refine_model, refine_optimizer, refine_scheduler, sdfs.unsqueeze(0), tets_verts, tets, f_vs, gt_verts, gt_faces, 0.001)

In [ ]:
surf_tets_verts_idx, surf_tets_faces = extract_tet(tets, sdfs, 0.001)
E = get_edges(surf_tets_faces).to(device)
refine_model(tets_verts, E)

# Run Surface Refinement

In [ ]:
refine_model = GCN_Res(CONFIG_GCNRES).to(device)

print(refine_model)

pre_vars = [p for _, p in refine_model.named_parameters()]
pre_optimizer = torch.optim.Adam(pre_vars, lr=lr)

In [ ]:
# pred_sdfs, _ = get_pred_sdfs(sdf_model, tets_verts)
pred, features = sdf_model(tets_verts)
pred_sdfs, deform = pred[:,0], pred[:,1:]

In [ ]:
gcn_pretrain(500, refine_model, pred_sdfs, tets_verts, tets,  0.001)
# gcn_pretrain(2000, refine_model)

In [ ]:
refine_vars = [p for _, p in refine_model.named_parameters()]
refine_optimizer = torch.optim.Adam(pre_vars, lr=lr)
refine_scheduler = torch.optim.lr_scheduler.LambdaLR(pre_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time

In [ ]:
torch.cuda.empty_cache()
gcn_train(gcn_iterations, refine_model, refine_optimizer, refine_scheduler, pred_sdfs, tets_verts, tets, features, gt_verts, gt_faces, 0)

# Ignore

In [ ]:
"""
to-dos: volume subdivision

identify T_surf's neighbors: i.e. share a same edge

subdivide T_surf to perform another surface refinement
unsubdivded tet, i.e. not surface's neighbor, is dropped to save memory and computation

"""

#neightbor if share an edge
#identify edge based on surf_faces and adj matrix

"""
convert face_lists = [v1, v2, v3, v4] --> 
[
  [v1, v2, v3, E, o1],
  [v2, v3, v4, E, o2],
  [v1, v2, v4, E, o3],
  [v1, v3, v4, E, o4]
]

by torch combinations, E = tet index, o_i = face_i index
"""
def convert_face_lists(tets):
  face_idx = torch.tensor([1,2,3,4])
  tet_face_list = []
  for idx, tet in enumerate(tets):
    tet_idx = torch.full(4, idx)
    tet_faces = torch.combinations(tet, r=3)
    tet_faces = torch.cat((tet_faces, tet_idx, face_idx), dim=1)
    tet_face_list.append(tet_faces)

  tet_face_list = torch.stack(tet_face_list, dim=0)

  def compare(face_1, face_2):
    o1 = face_1[0]-face_2[0]
    o2 = face_1[1]-face_2[1]
    o3 = face_1[2]-face_2[2]

    if o1 <= 0 or (o1 == 0 and o2 < 0) or (o1 == 0 and o2 == 0 and o3 < 0):
      return -1
    elif o1 == 0 and o2 == 0 and o3 == 0:
      return 0
    else: 
      return 1

  sorted(tet_face_list, cmp=compare)
  return tet_face_list

def get_neighbor(surf_tet_verts, surf_tets, tet_verts, tets):
  tet_face_list = convert_face_lists(tets)
  

surf_idx, surf = extract_tet(tets, pred_sdfs, 0.003)

print(surf_idx)

In [ ]:
def get_faces(input):
  c = torch.combinations(torch.arange(input.size(1)), r=3)
  x = input[:,None].expand(-1,len(c),-1).cpu()
  idx = c[None].expand(len(x), -1, -1)
  x = x.gather(dim=2, index=idx)

  return x.view(-1, *x.shape[2:])

In [ ]:
# a = get_faces(tets)
# b = get_faces(tets[surf_idx])

# # OPTIMIZE TF OUT OF THIS
# for f in b:
#   mask = a == f


# SDF Model and Interpolation



```
# This is formatted as code
```

Other idea is to use a Point Cloud Encoder to retrieve a feature vector from the output's activation.

In [ ]:
ENCODER_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 800, 1600, 1600],
    'output_dim' : 832, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
}

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()

        self.input_dim = config['input_dim']
        self.hidden_dims  = config['hidden_dims']
        self.output_dim = config['output_dim']

        # Hidden layers
        self.hiddens = nn.ModuleList()
        in_dim = self.input_dim
        for k in range(len(self.hidden_dims)):
            self.hiddens.append(nn.Linear(in_dim, self.hidden_dims[k]))
            in_dim = self.hidden_dims[k]

        # Output layer
        self.output_layer = torch.nn.Linear(self.hidden_dims[-1], self.output_dim)


    def forward(self, x):
        for hidden in self.hiddens :
            x = F.relu(hidden(x))
        output = self.output_layer(x) # No activation (linear) cuz we do regression

        return output, x # Return output + last feature layer vector

In [ ]:
# pointnet = MLP(ENCODER_MLP_CONFIG)
# pointnet = pointnet.to(device)

# pc = pc.to(device)

# print(pointnet)
# print('\n\n')
# summary(pointnet, input_size= pc.shape)

# Interpolate feature values on the grid

*We* implement an inverse distance interpolation based on a K-NN algorithm :
Given N known points and their features and a batch of M points with unknown features, the interpolator find the K nearest neigbors in the set N for each point of the batch M. Then it interpolates the features of the batch points M using an inverse distance weighting of the features of the K known neighbour points. 

K can be fine-tuned to balance efficiency and speed.

Implementation can be found here
https://stackoverflow.com/questions/3104781/inverse-distance-weighted-idw-interpolation-with-python

In [54]:
from __future__ import division
import numpy as np
from scipy.spatial import cKDTree as KDTree
    # http://docs.scipy.org/doc/scipy/reference/spatial.html

__date__ = "2010-11-09 Nov"  # weights, doc

#...............................................................................
class Invdisttree:
    """ inverse-distance-weighted interpolation using KDTree:
invdisttree = Invdisttree( X, z )  -- data points, values
interpol = invdisttree( q, nnear=3, eps=0, p=1, weights=None, stat=0 )
    interpolates z from the 3 points nearest each query point q;
    For example, interpol[ a query point q ]
    finds the 3 data points nearest q, at distances d1 d2 d3
    and returns the IDW average of the values z1 z2 z3
        (z1/d1 + z2/d2 + z3/d3)
        / (1/d1 + 1/d2 + 1/d3)
        = .55 z1 + .27 z2 + .18 z3  for distances 1 2 3

    q may be one point, or a batch of points.
    eps: approximate nearest, dist <= (1 + eps) * true nearest
    p: use 1 / distance**p
    weights: optional multipliers for 1 / distance**p, of the same shape as q
    stat: accumulate wsum, wn for average weights

How many nearest neighbors should one take ?
a) start with 8 11 14 .. 28 in 2d 3d 4d .. 10d; see Wendel's formula
b) make 3 runs with nnear= e.g. 6 8 10, and look at the results --
    |interpol 6 - interpol 8| etc., or |f - interpol*| if you have f(q).
    I find that runtimes don't increase much at all with nnear -- ymmv.

p=1, p=2 ?
    p=2 weights nearer points more, farther points less.
    In 2d, the circles around query points have areas ~ distance**2,
    so p=2 is inverse-area weighting. For example,
        (z1/area1 + z2/area2 + z3/area3)
        / (1/area1 + 1/area2 + 1/area3)
        = .74 z1 + .18 z2 + .08 z3  for distances 1 2 3
    Similarly, in 3d, p=3 is inverse-volume weighting.

Scaling:
    if different X coordinates measure different things, Euclidean distance
    can be way off.  For example, if X0 is in the range 0 to 1
    but X1 0 to 1000, the X1 distances will swamp X0;
    rescale the data, i.e. make X0.std() ~= X1.std() .

A nice property of IDW is that it's scale-free around query points:
if I have values z1 z2 z3 from 3 points at distances d1 d2 d3,
the IDW average
    (z1/d1 + z2/d2 + z3/d3)
    / (1/d1 + 1/d2 + 1/d3)
is the same for distances 1 2 3, or 10 20 30 -- only the ratios matter.
In contrast, the commonly-used Gaussian kernel exp( - (distance/h)**2 )
is exceedingly sensitive to distance and to h.

    """
# anykernel( dj / av dj ) is also scale-free
# error analysis, |f(x) - idw(x)| ? todo: regular grid, nnear ndim+1, 2*ndim

    def __init__( self, X, z, leafsize=10, stat=0 ):
        assert len(X) == len(z), "len(X) %d != len(z) %d" % (len(X), len(z))
        self.tree = KDTree( X, leafsize=leafsize )  # build the tree
        self.z = z
        self.stat = stat
        self.wn = 0
        self.wsum = None;

    def __call__( self, q, nnear=6, eps=0, p=1, weights=None ):
            # nnear nearest neighbours of each query point --
        q = np.asarray(q)
        qdim = q.ndim
        if qdim == 1:
            q = np.array([q])
        if self.wsum is None:
            self.wsum = np.zeros(nnear)

        self.distances, self.ix = self.tree.query( q, k=nnear, eps=eps )
        interpol = np.zeros( (len(self.distances),) + np.shape(self.z[0]) )
        jinterpol = 0
        for dist, ix in zip( self.distances, self.ix ):
            if nnear == 1:
                wz = self.z[ix]
            elif dist[0] < 1e-10:
                wz = self.z[ix[0]]
            else:  # weight z s by 1/dist --
                w = 1 / dist**p
                if weights is not None:
                    w *= weights[ix]  # >= 0
                w /= np.sum(w)
                wz = np.dot( w, self.z[ix] )
                if self.stat:
                    self.wn += 1
                    self.wsum += w
            interpol[jinterpol] = wz
            jinterpol += 1
        return interpol if qdim > 1  else interpol[0]

Let's test the interpolation by splitting the point cloud (of size 1000) into a 

1.   List item
2.   List item

known set of size N= 600 and an unknown batch of M= 400 points

In [44]:
INTERP_MLP_CONFIG = {
    'input_dim' : 3, # Coordinates of the grid's vertices
    'hidden_dims' : [256, 800, 1600, 1600],
    'output_dim' : 832, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
}


pointnet = MLP(INTERP_MLP_CONFIG).to(device)
print(pointnet)
print('\n\n')
summary(pointnet, input_size= points.shape)

MLP(
  (hiddens): ModuleList(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=800, bias=True)
    (2): Linear(in_features=800, out_features=1600, bias=True)
    (3): Linear(in_features=1600, out_features=1600, bias=True)
  )
  (output_layer): Linear(in_features=1600, out_features=832, bias=True)
)



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1            [-1, 5977, 256]           1,024
            Linear-2            [-1, 5977, 800]         205,600
            Linear-3           [-1, 5977, 1600]       1,281,600
            Linear-4           [-1, 5977, 1600]       2,561,600
            Linear-5            [-1, 5977, 832]       1,332,032
Total params: 5,381,856
Trainable params: 5,381,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.07
Forward/backward pass size (

In [60]:
pointnet = MLP(INTERP_MLP_CONFIG).to(device)
pointnet.pre_train_sphere(1000)
sdf_vars = [p for _, p in pointnet.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time
# ~12 min. Speed up?
encoder_train(iterations, sdf_model, sdf_optimizer, sdf_scheduler)

pointnet.eval()
f_vol, _ = pointnet(points)


sample_known = points.squeeze()[:600] # shape 600 x 3. Set of N= 600 points with known features
sample_unknown = points.squeeze()[600:] # shape 400x3. Batch of M= 400 points on which we will interpolate the features
f_vol_known = f_vol.squeeze()[:600] # Shape 600 x 832. Set of the feature vectors F_vol of the set N
f_vol_unknown = f_vol.squeeze()[600:] # Shape 400 x 832. Ground truth of F_vol for the batch M unknown points. Used to test the interpolation performance


Initialize SDF to sphere


100%|██████████| 1000/1000 [00:15<00:00, 65.49it/s]

Pre-trained MLP 7.087472795319627e-07


RuntimeError: ignored

In [39]:
# sample_known = pc.squeeze()[:600] # shape 600 x 3. Set of N= 600 points with known features
# sample_unknown = pc.squeeze()[600:] # shape 400x3. Batch of M= 400 points on which we will interpolate the features
# f_vol_known = f_vol.squeeze()[:600] # Shape 600 x 832. Set of the feature vectors F_vol of the set N
# f_vol_unknown = f_vol.squeeze()[600:] # Shape 400 x 832. Ground truth of F_vol for the batch M unknown points. Used to test the interpolation performance


leafsize = 10 # leaf size of the KDTree. This means the KDTree will store "leafsize" number of neighbour points for each data point
eps = .1  # approximate nearest, dist <= (1 + eps) * true nearest
p = 1  # weights ~ 1 / distance**p
Nnear = 8  # 8 2d, 11 3d => 5 % chance one-sided -- Wendel, mathoverflow.com


invdisttree = Invdisttree( sample_known.squeeze().detach().cpu(), f_vol_known.squeeze().detach().cpu(), leafsize=leafsize, stat=1 )
interpol = invdisttree( sample_unknown.detach().cpu(), nnear=Nnear, eps=eps, p=p ) # return numpy array

# err = np.abs( f_vol_unknown.detach().numpy() - interpol )
# print("average |ground_truth - interpolated|: %.2g" % np.mean(err))

Now we compute the F_vol interpolation on the grid vertices

In [50]:
tets_f_vol = invdisttree( tets_verts.detach().cpu(), nnear=Nnear, eps=eps, p=p ) # ~ 5-10sec
tets_f_vol = torch.tensor(tets_f_vol, device=device, dtype=torch.float)

In [51]:
sdf_input = torch.cat(tensors= (tets_f_vol, tets_verts), dim= 1)

# Next part, add this to MLP Encoder

In [57]:
SDF_MLP_CONFIG = {
    'input_dim' : 3 + 832, # Coordinates of the grid's vertices + F_vol dimension (concatenation)
    'hidden_dims' : [256, 256, 128, 64],
    'output_dim' : 1, # SDF of the vertex input. The other "output" f_v comes from the prior activation layer of dimension 64
}
sdf_model = MLP(SDF_MLP_CONFIG).to(device)
sdf_model.pre_train_sphere(2000)
sdf_vars = [p for _, p in sdf_model.named_parameters()]
sdf_optimizer = torch.optim.Adam(sdf_vars, lr=lr)
sdf_scheduler = torch.optim.lr_scheduler.LambdaLR(sdf_optimizer, lr_lambda=lambda x: max(0.0, 10**(-x*0.0002))) # LR decay over time
# ~12 min. Speed up?
encoder_train(iterations, sdf_model, sdf_optimizer, sdf_scheduler, tv=sdf_input)

Initialize SDF to sphere


100%|██████████| 2000/2000 [00:22<00:00, 90.52it/s]


Pre-trained MLP 0.35540616512298584
Iteration 0 - loss: 29.653545379638672
Iteration 100 - loss: 29.653545379638672
Iteration 200 - loss: 29.653545379638672


KeyboardInterrupt: ignored

In [ ]:
torch.cuda.empty_cache()

# pred_sdfs, f_vs = sdf_model(sdf_input)

# Visualization

In [ ]:
#Use pyngrok to access localhost:80 on Colab

!pip install pyngrok --quiet 
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Hzzzh94FgOXssVkSP5Yffz8uYg_By2RMDZLTPx1aXakhYfH"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
#generating a public url mapped to localhost 80
public_url = ngrok.connect(port=80, proto="http", options={"bind_tls": True, "local": True})
print("Tracking URL:", public_url)

In [ ]:
#Start Kaolin Dash3D on localhost:80 
!kaolin-dash3d --logdir=/content/drive/MyDrive/CV_DMTet/Logs --port=80